In [1]:
import pandas as pd
import numpy as np
from rake_nltk import Rake
from nltk.corpus import stopwords 


In [2]:
df = pd.read_csv('/Users/user/FINAL CAPSTONE/inprogress3.csv')

/Users/user/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df[['title','writer','genre','director','actors','plot','language','country']]
df.head()

,title,writer,genre,director,actors,plot,language,country
0,Toy Story,"John Lasseter (original story by), Pete Docter...","Animation, Adventure, Comedy, Family, Fantasy",John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",A cowboy doll is profoundly threatened and jea...,English,USA
1,Jumanji,"Jonathan Hensleigh (screenplay by), Greg Taylo...","Adventure, Comedy, Family, Fantasy",Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",When two kids find and play a magical board ga...,"English, French",USA
2,Grumpier Old Men,"Mark Steven Johnson (characters), Mark Steven ...","Comedy, Romance",Howard Deutch,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",John and Max resolve to save their beloved bai...,"English, Italian, German",USA
3,Waiting to Exhale,"Terry McMillan (novel), Terry McMillan (screen...","Comedy, Drama, Romance",Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin...","Based on Terry McMillan's novel, this film fol...",English,USA
4,Father of the Bride Part II,"Albert Hackett (screenplay ""Father's Little Di...","Comedy, Family, Romance",Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb...",George Banks must deal not only with the pregn...,English,USA


In [4]:
for column in df:
    df[column] = df[column].apply(lambda x: str(x))


In [5]:
df = df.head(50000)

In [6]:
# discarding the commas between the actors' full names
df['actors'] = df['actors'].map(lambda x: x.split(','))

# putting the genres in a list of words
df['genre'] = df['genre'].map(lambda x: x.lower().split(','))

df['director'] = df['director'].map(lambda x: x.split(' '))

# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in df.iterrows():
    row['actors'] = [x.lower().replace(' ','') for x in row['actors']]
    row['director'] = ''.join(row['director']).lower()

In [7]:
# Getting rid of information between () in writers
df['writer'] = df['writer'].str.replace(r"\(.*\)","")

# discarding the commas between writer's names
df['writer'] = df['writer'].map(lambda x: x.split(','))

# putting the countries in a list of words
df['country'] = df['country'].map(lambda x: x.lower().split(','))

# putting the languages in a list of words
df['language'] = df['language'].map(lambda x: x.lower().split(','))

# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in df.iterrows():
    row['writer'] = [x.lower().replace(' ','') for x in row['writer']]
    row['country'] = ''.join(row['country']).lower()
    row['language'] = ''.join(row['language']).lower()

In [8]:
df.head()

,title,writer,genre,director,actors,plot,language,country
0,Toy Story,[johnlasseter],"[animation, adventure, comedy, family, fan...",johnlasseter,"[tomhanks, timallen, donrickles, jimvarney]",A cowboy doll is profoundly threatened and jea...,english,usa
1,Jumanji,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",joejohnston,"[robinwilliams, jonathanhyde, kirstendunst, br...",When two kids find and play a magical board ga...,english french,usa
2,Grumpier Old Men,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",howarddeutch,"[waltermatthau, jacklemmon, sophialoren, ann-m...",John and Max resolve to save their beloved bai...,english italian german,usa
3,Waiting to Exhale,[terrymcmillan],"[comedy, drama, romance]",forestwhitaker,"[whitneyhouston, angelabassett, lorettadevine,...","Based on Terry McMillan's novel, this film fol...",english,usa
4,Father of the Bride Part II,[alberthackett],"[comedy, family, romance]",charlesshyer,"[stevemartin, dianekeaton, martinshort, kimber...",George Banks must deal not only with the pregn...,english,usa


In [9]:
# initializing the new column
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df.drop(columns = ['plot'], inplace = True)

In [10]:
df.set_index('title', inplace = True)
df.head()

,writer,genre,director,actors,language,country,Key_words
title,,,,,,,
Toy Story,[johnlasseter],"[animation, adventure, comedy, family, fan...",johnlasseter,"[tomhanks, timallen, donrickles, jimvarney]",english,usa,"[cowboy, doll, room, top, toy, jealous, profou..."
Jumanji,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",joejohnston,"[robinwilliams, jonathanhyde, kirstendunst, br...",english french,usa,"[stopped, release, two, kids, find, man, trapp..."
Grumpier Old Men,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",howarddeutch,"[waltermatthau, jacklemmon, sophialoren, ann-m...",english italian german,usa,"[save, beloved, bait, shop, john, new, female,..."
Waiting to Exhale,[terrymcmillan],"[comedy, drama, romance]",forestwhitaker,"[whitneyhouston, angelabassett, lorettadevine,...",english,usa,"[based, american, women, different, african, r..."
Father of the Bride Part II,[alberthackett],"[comedy, family, romance]",charlesshyer,"[stevemartin, dianekeaton, martinshort, kimber...",english,usa,"[also, pregnancy, unexpected, george, banks, m..."


In [11]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)
indices[:5]

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object

In [13]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.15689291, 0.07548514, ..., 0.        , 0.        ,
        0.        ],
       [0.15689291, 1.        , 0.03849002, ..., 0.        , 0.        ,
        0.        ],
       [0.07548514, 0.03849002, 1.        , ..., 0.        , 0.03456506,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.03745029,
        0.05783149],
       [0.        , 0.        , 0.03456506, ..., 0.03745029, 1.        ,
        0.04981355],
       [0.        , 0.        , 0.        , ..., 0.05783149, 0.04981355,
        1.        ]])

In [14]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [18]:
recommendations('Amelie')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [19]:
from sklearn.pipeline import Pipeline

In [23]:
movie_recommender = Pipeline([('tf-idf', CountVectorizer()),
                              ('cosine_sim', cosine_similarity(count_matrix, count_matrix))], memory=None, verbose=1)



TypeError: __init__() got an unexpected keyword argument 'verbose'